create parameter

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')

### creating dimension Model

In [0]:
df_src = spark.sql('''
                   select DISTINCT(Dealer_ID) as Dealer_ID, DealerName from parquet.`abfss://silver@datastorager.dfs.core.windows.net/car_sales`
                   
                   ''')


In [0]:
df_src.show()

In [0]:
# Check if the target table exists
if spark.catalog.tableExists('cars_catalog.gold_schema.dim_Dealer'):

    # Read existing dimension data from the source parquet file
    df_sink = spark.sql('''
    select  dim_Dealer_key, Dealer_ID, DealerName from cars_catalog.gold_schema.dim_Dealer
                        ''')

else:
    # Create an empty DataFrame with the required schema if the table does not exist
    df_sink = spark.sql('''
select 1 as dim_Dealer_key, Dealer_ID, DealerName from parquet.`abfss://silver@datastorager.dfs.core.windows.net/car_sales` where 1=0
                    ''')

In [0]:
df_sink.show()

In [0]:
df_filter = df_src.join(df_sink, df_src.Dealer_ID == df_sink.Dealer_ID, 'left').select(df_src.Dealer_ID, df_src.DealerName, df_sink.dim_Dealer_key)

In [0]:
df_filter.show()

In [0]:
df_filter_old = df_filter.filter(col('dim_Dealer_key').isNotNull())
df_filter_old.show()

In [0]:
df_filter_new = df_filter.filter(df_filter.dim_Dealer_key.isNull()).select(df_filter.Dealer_ID, df_filter.DealerName)
df_filter_new.show()

create surrogate key

In [0]:
if incremental_flag == "0":
    max_value = 1
else:
    max_value_df = spark.sql('''select max(dim_Dealer_key) from cars_catalog.gold_schema.dim_Dealer''')
    max_value =max_value_df.collect()[0][0]

In [0]:
df_filter_new = df_filter_new.withColumn('dim_Dealer_key', max_value+monotonically_increasing_id()) 

In [0]:
 df_filter_new.show()

In [0]:
df_final = df_filter_old.union(df_filter_new)
df_final.show()


## SCD1-upsert()

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists('cars_catalog.gold_schema.dim_Dealer'):
    print("if")
    delta_tbl = DeltaTable.forPath(spark,'abfss://gold@datastorager.dfs.core.windows.net/dim_Dealer')
    delta_tbl.alias('trg').merge(df_final.alias('src'), 'trg.dim_Dealer_key = src.dim_Dealer_key').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

#initial run
else:
    print("else")
    df_final.write.format('delta').mode('overwrite').option('path','abfss://gold@datastorager.dfs.core.windows.net/dim_Dealer').saveAsTable('cars_catalog.gold_schema.dim_Dealer')

In [0]:
%sql
select * from cars_catalog.gold_schema.dim_Dealer